In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

https://www.youtube.com/watch?v=1-NYPQw5THU&feature=youtu.be

In [2]:
import pandas as pd
import numpy as np
import datetime
from pandas_summary import DataFrameSummary

In [3]:
df = pd.read_feather('train_normalized_data.fth')
df_test = pd.read_feather('test_normalized_data.fth')

In [4]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

# cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'State']


In [5]:
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']
# contin_vars = []

In [6]:
from lightgbm import LGBMRegressor

In [7]:
y_out_columns = ['Sales']

In [8]:
# split_type = 'random'
# split_type = 'no_split'
split_type = 'last_week'

In [9]:
# Esta es para entrenar con todo
if split_type == 'no_split':
    df_train = df
elif split_type == 'last_week':
    # Esto divide en train y val
    df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
    df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
    print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')
elif split_type == 'random':
    # Splitting aleatorio
    np.random.seed(42)
    indexes = np.arange(len(df))
    np.random.shuffle(indexes)
    N = len(df)//5
    df_train = df[N:]
    df_val = df[:N]
    print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')

Cantidad en val: 30188, porcentaje: 0.9642465458145908


In [10]:
X_train = df_train[cat_vars + contin_vars]
if split_type != 'no_split':
    X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

In [11]:
log_output = True
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    if split_type != 'no_split':
        y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    if split_type != 'no_split':
        y_val = (df_val[y_out_columns].values - y_mean)/y_std

In [12]:
from sklearn.model_selection import cross_val_score
from hyperopt import hp, tpe
from hyperopt.fmin import fmin

In [13]:
def calculate_RMSE(X, y, log_output=True):
    y_preds = np.exp(model.predict(X, verbose=1)*max_log_y)
    return np.sqrt((((y - y_preds)/y)**2).sum()/len(y_preds))

In [14]:
X_train.shape

(814150, 38)

In [32]:
def objective(params):
    params = {
        # 'max_depth': int(params['max_depth']),
        # 'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'learning_rate': '{:.4f}'.format(params['learning_rate'])
    }
    
    clf = LGBMRegressor(
        n_estimators=4000,
        min_child_samples= 5, max_depth= -1,
        **params
    )
    
    score = -cross_val_score(clf, X_train.values, y_train.reshape(-1)).mean()
    return score

In [33]:
space = {
    # 'max_depth': hp.quniform('max_depth', 1, 64, 4),
    # 'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'learning_rate': hp.loguniform('learning_rate', -4, -2)
}

best = fmin(fn=objective,
            space=space,
            verbose=2,
            algo=tpe.suggest,
            max_evals=10)

  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/home/usuario/anaconda3/envs/GPUV2/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)



 10%|█         | 1/10 [02:23<21:32, 143.63s/it, best loss: -0.896349713054469]

/home/usuario/anaconda3/envs/GPUV2/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)



 20%|██        | 2/10 [05:03<19:47, 148.39s/it, best loss: -0.896349713054469]

...

 80%|████████  | 8/10 [22:33<05:50, 175.00s/it, best loss: -0.896349713054469]

/home/usuario/anaconda3/envs/GPUV2/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)



 90%|█████████ | 9/10 [26:27<03:12, 192.61s/it, best loss: -0.896349713054469]

/home/usuario/anaconda3/envs/GPUV2/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)



100%|██████████| 10/10 [29:12<00:00, 184.29s/it, best loss: -0.896349713054469]


In [34]:
best

{'learning_rate': 0.10054532290551892}

In [13]:
from sklearn.model_selection import GridSearchCV

In [36]:
parameters = {'learning_rate':[0.001, 0.05, 0.1, 0.5], 'max_depth':[1, 7, 31, 63]}
grid_search_res_1 = {'learning_rate': 0.05, 'min_child_samples': 5, 'max_depth': -1}
grid_search_res_2 = {'learning_rate': 0.05, 'max_depth': 7, 'min_child_samples': 20}

In [37]:
min_child_samples = 5
n_estimators = 4000
learning_rate = 0.1
model = LGBMRegressor(n_estimators=n_estimators, **grid_search_res_2)

In [38]:
clf = GridSearchCV(model, parameters, n_jobs=-1, verbose=1, cv=3)

NameError: name 'GridSearchCV' is not defined

In [41]:
fit_params={ "eval_metric" : 'l2', 
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': cat_vars}
if split_type != 'no_split':
    fit_params["eval_set"] = [(X_val, y_val.reshape(-1))]
    fit_params["early_stopping_rounds"] = 100

In [42]:
# clf.fit(X_train, y_train.reshape(-1), **fit_params)
model.fit(X_train, y_train.reshape(-1), **fit_params)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.000278838	valid_0's l2: 0.000278838
[200]	valid_0's l2: 0.000205993	valid_0's l2: 0.000205993
[300]	valid_0's l2: 0.000166847	valid_0's l2: 0.000166847
[400]	valid_0's l2: 0.000148417	valid_0's l2: 0.000148417
[500]	valid_0's l2: 0.000137431	valid_0's l2: 0.000137431
[600]	valid_0's l2: 0.000130973	valid_0's l2: 0.000130973
[700]	valid_0's l2: 0.000125796	valid_0's l2: 0.000125796
[800]	valid_0's l2: 0.0001229	valid_0's l2: 0.0001229
[900]	valid_0's l2: 0.000120087	valid_0's l2: 0.000120087
[1000]	valid_0's l2: 0.000118255	valid_0's l2: 0.000118255
[1100]	valid_0's l2: 0.000116602	valid_0's l2: 0.000116602
[1200]	valid_0's l2: 0.000115523	valid_0's l2: 0.000115523
[1300]	valid_0's l2: 0.000114763	valid_0's l2: 0.000114763
[1400]	valid_0's l2: 0.000113678	valid_0's l2: 0.000113678
[1500]	valid_0's l2: 0.000113048	valid_0's l2: 0.000113048
[1600]	valid_0's l2: 0.000112622	valid_0's l2: 0.000112622
[1700]

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.05, max_depth=7,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=4000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [43]:
clf

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split',
                                     learning_rate=0.05, max_depth=7,
                                     min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=4000, n_jobs=-1,
                                     num_leaves=31, objective=None,
                                     random_state=None, reg_alpha=0.0,
                                     reg_lambda=0.0, silent=True, subsample=1.0,
                                     subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='warn', n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.05, 0.1, 0.5],
                         'max_

In [44]:
# clf.best_params_

In [45]:
# clf.best_score_

# Métrica

$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [53]:
model.score(X_val, y_val)

0.9231718729428571

In [47]:
if log_output:
    y_pred_train = np.exp(model.predict(X_train, verbose=1)*max_log_y)
    y_pred = np.exp(model.predict(X_val, verbose=1)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test, verbose=1)*max_log_y)
else:
    y_pred_train = model.predict(X_train, verbose=1)*y_std + y_mean
    y_pred = model.predict(X_val, verbose=1)*y_std + y_mean
    y_pred_test = model.predict(X_test, verbose=1)*y_std + y_mean

In [48]:
# Train
np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train))

0.08101860769764169

In [55]:
# Validación
np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))

0.11929361858557005

In [57]:
calculate_RMSE(X_train, df_train['Sales'].values)

0.08101860769764169

In [58]:
calculate_RMSE(X_val, df_val['Sales'].values)

0.11929361858557005

# Sumbit a la competición

In [50]:
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

sample_csv.to_csv(f'submision_lightgbm_{split_type}-{log_output}-{min_child_samples}-{n_estimators}-{learning_rate}.csv', index=False)